In [1]:
%load_ext cython

In [74]:
%%cython
import random
import numpy as np
cimport numpy as np
np.import_array()

import collections
from mpire import WorkerPool
from decimal import Decimal

cdef list decks
cdef short deck_len, half_deck_len
cdef double parameters_0[3]
cdef int games_to_play

deck = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]*4  # deck of 52 cards
deck_len = len(deck)
half_deck_len = int(deck_len/2)
parameters_0 = [1, 1, 1]

cdef class Player():
    """
    A class that represents a player's basic moves
    """
    cdef list hand, cards
    cdef char card
    cdef double strategy[3]
    cdef np.ndarray parameters

    def __init__(self, parameters, cards = None):
        if cards is None:
            self.hand = [0]*52
        else:
            self.hand = cards
        self.strategy = np.exp(parameters , dtype = np.float128)
        # self.strategy = [np.exp(i, dtype=np.float128) for i in parameters]
    def confert_to_p(x):
        p = np.exp(x, dtype=np.float128)
        return p/np.sum(p)
    
    def get_card(self) :
        return self.hand.pop(0)

    def add_card(self, card):
        self.hand.append(card)

    def add_cards(self, cards):
        for card in cards:
            self.add_card(card)

    def get_hend_len(self):
        return len(self.get_hand())
    
    def get_hand(self) -> list:
        return self.hand
    
    def set_hend(self,cards):
        self.hand = cards
        
cdef class war3_player(Player):
    cdef char choice
    def make_decision(self, cards):
        choice = random.choices([1, 2, 3], weights = self.strategy)[0]
        if choice == 1: 
            cards = sorted(cards)
        elif choice == 2:
            cards = sorted(cards, reverse=True)
        else:
            random.shuffle(cards)
            
        self.add_cards(cards)
        
def battle(player_1, player_2):
    cdef char player_1_card, player_2_card, winner, player_1_card_1, player_2_card_1, player_1_card_2, player_2_card_2
    cdef list cards
    
    player_1_card = player_1.get_card()
    player_2_card = player_2.get_card()
    cards = [player_1_card, player_2_card]
    if player_1_card > player_2_card:
        winner = 1
        return winner, cards
    elif player_2_card > player_1_card:
        winner = 2
        return winner, cards
    else:
        while 1:
            player_1_card_1 = player_1.get_card()
            player_2_card_1 = player_2.get_card()
            player_1_card_2 = player_1.get_card()
            player_2_card_2 = player_2.get_card()
            cards.extend([player_1_card_1, player_2_card_1,
                        player_1_card_2,player_2_card_2])
            if player_1_card_2 > player_2_card_2:
                winner = 1
                return winner, cards
            elif player_2_card_2 > player_1_card_2:
                winner = 2
                return winner, cards


            
def symulation(parameters1, parameters2, deck = deck, k =0):
    cdef char battle_winner
    cdef list cards
    cdef list shuffle_decl
    np.random.seed()
    shuffle_decl = random.sample(deck, deck_len)
    player_1 = war3_player(parameters1)
    player_2 = war3_player(parameters2)
    player_1.set_hend(shuffle_decl[0:half_deck_len])
    player_2.set_hend(shuffle_decl[half_deck_len:deck_len])
    i=0
    while 1:
        i+=1
        try:
            battle_winner, cards = battle(player_1, player_2)
            if battle_winner == 1:
                player_1.make_decision(cards)
            else:
                player_2.make_decision(cards)
        except:
            if player_1.get_hend_len() == 0:
                return 1
            else:
                return 0
        if i > 20000:
            shuffle_decl = random.sample(deck, deck_len)
            player_1.set_hend(shuffle_decl[0:half_deck_len])
            player_2.set_hend(shuffle_decl[half_deck_len:deck_len])
            i = 0

In file included from /home/olek/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1948,
                 from /home/olek/.local/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/olek/.local/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from /home/olek/.cache/ipython/cython/_cython_magic_0b4e367d1d652ea60301ac86b770513d.c:778:
/home/olek/.local/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~


In [79]:
%%timeit 
symulation(np.array([0,1,0]), np.array([1,0,0]))

[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.718281828459045, 1.0, 1.0]
[1.0, 2.718281828459045, 1.0]
[2.7182818

In [80]:
from war import symulation

In [82]:
%%timeit 
symulation(np.array([0,1,0]),np.array([1,0,0]))                                                                   

1.64 ms ± 328 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
